# Building a model that can assist in scaling up the implementation of NPIs based on kalman filter and machine learning algorithms.

As we know the **Covid19** is frightening all of us and has affected more than 185 countries, there is an urgent need in ways to control its further spread and help people saving lives. There is a lot of research work that is being done currently in the field of effectiveness of different mitigation measures for **Covid19**. In this notebook we describe an effective way that can be used to deploy npis in regions based on various parameters. We hope this work would help us fighting the covid19 pandemic in an effective way.

This map shows the present status of the coronavirus infections across the world.

In [ ]:
from ipywidgets import Image
f = open("../input/covidimages/c1.jpg", "rb")
image = f.read()
Image(value=image)

# Goal/Tasks

1) Our major goal is to provide guidance on ways to scale up NPIs in a more coordinated way (e.g., establish funding, infrastructure and authorities to support real time, authoritative (qualified participants) collaboration with all states to gain consensus on consistent guidance and to mobilize resources to geographic areas where critical shortfalls are identified) to give us time to enhance our health care delivery system capacity to respond to an increase in cases.

2) Rapid assessment of the likely efficacy of school closures, travel bans, bans on mass gatherings of various sizes, and other social distancing approaches.

3) Models of potential interventions to predict costs and benefits that take account of such factors as race, income, disability, age, geographic location, immigration status, housing status, employment status, and health insurance status.

4) The last point would be building a guideline model that will help control, any such pandemic and not let things go too far.

# What all did we do?

1) First step: Prediction of the outbreak, forecasting in terms of number of cases, number of deaths etc.

2) Second step: Prediction of the best-suited npi for the particular situation based on the previous history of implementations.


* **Methods used for the first step:** We used regression techniques accompanied by a tool called Kalman filter which is found effective to give accurate predictions for such outbreaks.

* Kalman filter: In statistics and control theory, Kalman filtering, also known as linear quadratic estimation (LQE), is an algorithm that uses a series of measurements observed over time, containing statistical noise and other inaccuracies, and produces estimates of unknown variables that tend to be more accurate than those based on a single measurement alone, by estimating a joint probability distribution over the variables for each timeframe. The filter is named after Rudolf E. Kálmán, one of the primary developers of its theory.

* Source: https://en.wikipedia.org/wiki/Kalman_filter

* **Methods used for the second step** We have techniques like SMOTE(synthetic minority oversampling technique) to tackle our data-imbalance problem, various machine learning techniques like Random-Forest, KNearNeighboursClassifier, Decision trees etc. We have shown the performance of some of the best machine-learning techniques on our dataset in step two.

# Internet must be turned on for this notebook to work.

**Importing the necessary packages and libraries that are necessary for step-one.**

In [ ]:
import numpy as np
from math import sqrt
from sklearn.metrics import mean_squared_error
import pandas as pd
import cv2


* All the time-series data that is required for our analysis and prediction is taken from: https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset

* Although similar data is being maintained by this github repository: https://github.com/CSSEGISandData/COVID-19

* We will be going ahead with the data that is maintained by the above-mentioned **repository**.

* **Disclaimer**: Turn your internet on for this.

* Using the urls, we import the required data and store it different dataframes.

* We are using the **Pandas** library for doing this task.

* You can read the **official-documentation** for any additional information: https://pandas.pydata.org/

In [ ]:

url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
confirmed = pd.read_csv(url, error_bad_lines=False)
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
death = pd.read_csv(url, error_bad_lines=False)
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
recover = pd.read_csv(url, error_bad_lines=False)

# Replacing some of the column names.

In [ ]:
# replacing the region names
confirmed['Country/Region']= confirmed['Country/Region'].str.replace("Mainland China", "China")
confirmed['Country/Region']= confirmed['Country/Region'].str.replace("US", "Unites States")
death['Country/Region']= death['Country/Region'].str.replace("Mainland China", "China")
death['Country/Region']= death['Country/Region'].str.replace("US", "Unites States")
recover['Country/Region']= recover['Country/Region'].str.replace("Mainland China", "China")
recover['Country/Region']= recover['Country/Region'].str.replace("US", "Unites States")

* In the next step, we are going to merge the population-values of different countries with all the above created dataframes.

* The population-values are being taken from this dataset on kaggle: https://www.kaggle.com/fernandol/countries-of-the-world

In [ ]:
population=pd.read_csv('../input/populationv/population.csv', sep=',', encoding='latin1') 
confirmed=pd.merge(confirmed, population,how='left' ,on=['Province/State','Country/Region'])
death=pd.merge(death, population,how='left' ,on=['Province/State','Country/Region'])
recover=pd.merge(recover, population,how='left' ,on=['Province/State','Country/Region'])

* We make sure this is working by looking at the first 5 rows of the dataframe, by calling the pandas.dataframe.head() function.

* You can find more information regarding this function here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html

In [ ]:
confirmed.head()

In [ ]:
#now we merge the regions
confirmed['region']=confirmed['Country/Region'].map(str)+'_'+confirmed['Province/State'].map(str)
death['region']=death['Country/Region'].map(str)+'_'+death['Province/State'].map(str)
recover['region']=recover['Country/Region'].map(str)+'_'+recover['Province/State'].map(str)
confirmed=confirmed[confirmed.iloc[:,confirmed.shape[1]-3]>1000]# regions having more than 1000 active cases till date.
confirmed.iloc[:5,:]

* Now we extract the data in the form of time-series with the help the below displayed- create_ab function which accepts the dataframe as its parameter and returns a dataframe.

* Along with it we drop certain columns such as Lat(i.e the latitude), as we dont have any use of them at this step.

In [ ]:
#a function to just extract the time series information excluding other columns.
def create_ab(df):
  ab=df
  ab=ab.drop(['Province/State', 'Country/Region','Lat', 'Long',' Population '], axis=1)
  ab.set_index('region')
  ab=ab.T
  ab.columns=ab.loc['region']
  ab=ab.drop('region')
  ab=ab.fillna(0)
  ab=ab.reindex(sorted(ab.columns), axis=1)
  return (ab)

In [ ]:
#creating time series data of confirmed, deaths, recovered cases.
ts=create_ab(confirmed)
ts_d=create_ab(death)
ts_rec=create_ab(recover)

* Again we use the pd.DataFrame.head() to take a peek!

In [ ]:
ts.head()

# Plotting time-series data
* Based on whatever time-series data we created, now we are plotting it with the help of Matplotlib library.

* You can find more information regarding matplotlib here: https://matplotlib.org/

In [ ]:
import matplotlib.pyplot as plt
p=ts.reindex(ts.max().sort_values(ascending=False).index, axis=1)
p.iloc[:,:1].plot(marker='*',figsize=(10,4)).set_title('Daily Total Confirmed - UNITED STATES OF AMERICA',fontdict={'fontsize': 22})
p.iloc[:,2:10].plot(marker='*',figsize=(10,4)).set_title('Daily Total Confirmed - Major areas',fontdict={'fontsize': 22})

p_d=ts_d.reindex(ts.mean().sort_values(ascending=False).index, axis=1)
p_d.iloc[:,:1].plot(marker='*',figsize=(10,4)).set_title('Daily Total Death - Hubei',fontdict={'fontsize': 22})
p_d.iloc[:,2:10].plot(marker='*',figsize=(10,4)).set_title('Daily Total Death - Major areas',fontdict={'fontsize': 22})

p_r=ts_rec.reindex(ts.mean().sort_values(ascending=False).index, axis=1)
p_r.iloc[:,:1].plot(marker='*',figsize=(10,4)).set_title('Daily Total Recoverd - Hubei',fontdict={'fontsize': 22})
p_r.iloc[:,2:10].plot(marker='*',figsize=(10,4)).set_title('Daily Total Recoverd - Major areas',fontdict={'fontsize': 22})

* Now, saving the ts_r dataframe to our directory, by using the pd.to_csv() function.

In [ ]:
ts_r=ts.reset_index()
ts_r=ts_r.rename(columns = {'index':'date'})
ts_r['date']=pd.to_datetime(ts_r['date'] ,errors ='coerce')
#ts_r.to_csv(r'../input/populationv/ts_r2.csv')

# Error in importing rpy2 package.
* I was trying to import the rpy2 package where in R can be used in python kernels, but there was an error for me on kaggle.

* So im just writing the code here but not actually executing it, you can check my github repository here, for detailed output.

* I will save all the necessary files for the step2 (npi prediction) and import them again during that.

* The rpy2 package helps in using R in python kernels.

* You can find more information here: https://rpy2.github.io/doc/latest/html/introduction.html

In [ ]:
#importing rpy2 package for using R.
import rpy2
%load_ext rpy2.ipython

* Installing pracma and reshape packages.

* You can find more information regarding pracma here: https://cran.r-project.org/package=pracma

* You can find more information regarding reshape here: https://cran.r-project.org/package=reshape

In [ ]:
%%R
install.packages('pracma')
install.packages('reshape')

In [0]:
%%R
require(pracma)
require(Metrics)
require(readr)
all<- read_csv("../input/population/ts_r2.csv")
all$X1<-NULL
date<-all[,1]
date[nrow(date) + 1,1] <-all[nrow(all),1]+1
pred_all<-NULL
for (n in 2:ncol(all)-1) {
  Y<-ts(data = all[n+1], start = 1, end =nrow(all)+1)  
  sig_w<-0.01
  w<-sig_w*randn(1,100) # acceleration which denotes the fluctuation (Q/R) rnorm(100, mean = 0, sd = 1)
  sig_v<-0.01
  v<-sig_v*randn(1,100)   
  t<-0.45
  phi<-matrix(c(1,0,t,1),2,2)
  gama<-matrix(c(0.5*t^2,t),2,1)
  H<-matrix(c(1,0),1,2)
  #Kalman
  x0_0<-p0_0<-matrix(c(0,0),2,1)
  p0_0<-matrix(c(1,0,0,1),2,2)
  Q<-0.01
  R<-0.01
  X<-NULL
  X2<-NULL
  pred<-NULL
  for (i in 0:nrow(all)) {
    namp <-paste("p", i+1,"_",i, sep = "")
    assign(namp, phi%*%(get(paste("p", i,"_",i, sep = "")))%*%t(phi)+gama%*%Q%*%t(gama))
    namk <- paste("k", i+1, sep = "")
    assign(namk,get(paste("p", i+1,"_",i, sep = ""))%*%t(H)%*%(1/(H%*%get(paste("p", i+1,"_",i, sep = ""))%*%t(H)+R)))
    namx <- paste("x", i+1,"_",i, sep = "")
    assign(namx,phi%*%get(paste("x", i,"_",i, sep = "")))
    namE <- paste("E", i+1, sep = "")
    assign(namE,Y[i+1]-H%*%get(paste("x", i+1,"_",i, sep = "")))
    namx2 <- paste("x", i+1,"_",i+1, sep = "")
    assign(namx2,get(paste("x", i+1,"_",i, sep = ""))+get(paste("k", i+1, sep = ""))%*%get(paste("E", i+1, sep = "")))
    namp2 <- paste("p", i+1,"_",i+1, sep = "")
    assign(namp2,(p0_0-get(paste("k", i+1, sep = ""))%*%H)%*%get(paste("p", i+1,"_",i, sep = "")))
    X<-rbind(X,get(paste("x", i+1,"_",i,sep = ""))[1])
    X2<-rbind(X2,get(paste("x", i+1,"_",i,sep = ""))[2])
    if(i>2){
      remove(list=(paste("p", i-1,"_",i-2, sep = "")))
      remove(list=(paste("k", i-1, sep = "")))
      remove(list=(paste("E", i-1, sep = "")))
      remove(list=(paste("p", i-2,"_",i-2, sep = "")))
      remove(list=(paste("x", i-1,"_",i-2, sep = "")))
      remove(list=(paste("x", i-2,"_",i-2, sep = "")))}
  }
  pred<-NULL
  pred<-cbind(Y,X,round(X2,4))
  pred<-as.data.frame(pred)
  pred$region<-colnames(all[,n+1])
  pred$date<-date$date
  pred$actual<-rbind(0,(cbind(pred[2:nrow(pred),1])/pred[1:nrow(pred)-1,1]-1)*100)
  pred$predict<-rbind(0,(cbind(pred[2:nrow(pred),2])/pred[1:nrow(pred)-1,2]-1)*100)
  pred$pred_rate<-(pred$X/pred$Y-1)*100
  pred$X2_change<-rbind(0,(cbind(pred[2:nrow(pred),3]-pred[1:nrow(pred)-1,3])))
  pred_all<-rbind(pred_all,pred)
}
pred_all<-cbind(pred_all[,4:5],pred_all[,1:3])
names(pred_all)[5]<-"X2"
pred_all=pred_all[with( pred_all, order(region, date)), ]
pred_all<-pred_all[,3:5]

In [0]:
p=%R pred_all


# Merging the R ouput due to the package problem.

In [0]:

t=ts
t=t.stack().reset_index(name='confirmed')
t.columns=['date', 'region','confirmed']
t['date']=pd.to_datetime(t['date'] ,errors ='coerce')
t=t.sort_values(['region', 'date'])

temp=t.iloc[:,:3]
temp=temp.reset_index(drop=True)
for i in range(1,len(t)+1):
 if(temp.iloc[i,1] is not temp.iloc[i-1,1]):
    temp.loc[len(temp)+1] = [temp.iloc[i-1,0]+ pd.DateOffset(1),temp.iloc[i-1,1], 0] 
temp=temp.sort_values(['region', 'date'])
p.set_index(temp.index,inplace=True)
#temp=temp.reset_index(drop=True)
temp['Y']=p['Y']
temp['X']=p['X']
temp['X2']=p['X2']


* For details regarding minimum and maximum temperatures, reference was taken from the dataset used in this kernel: https://www.kaggle.com/davidbnn92/weather-data/data

In [ ]:
w=pd.read_csv('../input/weather/w.csv', sep=',', encoding='latin1')
w['date']=pd.to_datetime(w['date'],format='%d/%m/%Y')
#w['date']=pd.to_datetime(w['date'],errors ='coerce')

w_forecast=pd.read_csv('../input/weather/w_forecast.csv', sep=',', encoding='latin1')
w_forecast['date']=pd.to_datetime(w_forecast['date'],format='%d/%m/%Y')

In [ ]:
t=ts
t=t.stack().reset_index(name='confirmed')
t.columns=['date', 'region','confirmed']
t['date']=pd.to_datetime(t['date'] ,errors ='coerce')
t=t.sort_values(['region', 'date'])

# Add 1 Future day for prediction
t=t.reset_index(drop=True)
for i in range(1,len(t)+1):
  if(t.iloc[i,1] is not t.iloc[i-1,1]):
    t.loc[len(t)+1] = [t.iloc[i-1,0]+ pd.DateOffset(1),t.iloc[i-1,1], 0] 
t=t.sort_values(['region', 'date'])
t=t.reset_index(drop=True)

# Training our regression algorithm.

In [ ]:
t['1_day_change']=t['3_day_change']=t['7_day_change']=t['1_day_change_rate']=t['3_day_change_rate']=t['7_day_change_rate']=t['last_day']=0
for i in range(1,len(t)):
  if(t.iloc[i,1] is t.iloc[i-2,1]):
    t.iloc[i,3]=t.iloc[i-1,2]-t.iloc[i-2,2]
    t.iloc[i,6]=(t.iloc[i-1,2]/t.iloc[i-2,2]-1)*100
    t.iloc[i,9]=t.iloc[i-1,2]
  if(t.iloc[i,1] is t.iloc[i-4,1]):
    t.iloc[i,4]=t.iloc[i-1,2]-t.iloc[i-4,2]
    t.iloc[i,7]=(t.iloc[i-1,2]/t.iloc[i-4,2]-1)*100
  if(t.iloc[i,1] is t.iloc[i-8,1]):
    t.iloc[i,5]=t.iloc[i-1,2]-t.iloc[i-8,2]
    t.iloc[i,8]=(t.iloc[i-1,2]/t.iloc[i-8,2]-1)*100
t=t.fillna(0)  
t=t.merge(temp[['date','region', 'X']],how='left',on=['date','region'])
t=t.rename(columns = {'X':'kalman_prediction'}) 
t=t.replace([np.inf, -np.inf], 0)
t['kalman_prediction']=round(t['kalman_prediction'])
train=t.merge(confirmed[['region',' Population ']],how='left',on='region')
train=train.rename(columns = {' Population ':'population'})
train['population']=train['population'].str.replace(r" ", '')
train['population']=train['population'].str.replace(r",", '')
train['population']=train['population'].fillna(1)
train['population']=train['population'].astype('int32')
train['infected_rate'] =train['last_day']/train['population']*10000
train=train.merge(w,how='left',on=['date','region'])
train=train.sort_values(['region', 'date'])
### fill missing weather 
for i in range(0,len(train)):
    if(np.isnan(train.iloc[i,13])):
        if(train.iloc[i,1] is train.iloc[i-1,1]):
            train.iloc[i,13]=train.iloc[i-1,13]
            train.iloc[i,14]=train.iloc[i-1,14]

In [0]:
# Select region
region='China_Hubei'
place=0

evaluation=pd.DataFrame(columns=['region','mse','rmse','mae'])
for i in range(1,len(t)):
    if(t.iloc[i,1] is not t.iloc[i-1,1]):
        ex=np.array(t.iloc[i-len(ts):i,10])
        pred=np.array(t.iloc[i-len(ts):i,2])
        evaluation=evaluation.append({'region': t.iloc[i-1,1], 'mse': np.power((ex - pred),2).mean(),'rmse':sqrt(mean_squared_error(ex,pred)),'mae': (abs(ex - pred)).mean()}, ignore_index=True)
p=t[t['region']==region][['date','region','confirmed','kalman_prediction']]
p=p.rename(columns = {'confirmed':'confirmedd'})
p.iloc[len(p)-1,2]=None
p=p.set_index(['date'])
p.iloc[:,1:].plot(marker='o',figsize=(16,8)).set_title('Kalman Prediction - Select Region to Change - {}'.format(p.iloc[0,0]))
print(evaluation[evaluation['region']==p.iloc[0,0]])


* Displaying the output for the above cell, by executing the code on jupyter-notebook. 

In [ ]:
from ipywidgets import Image
f = open("../input/graphs/g1.png", "rb")
image = f.read()
Image(value=image)

In [0]:
plt.style.use('ggplot')
# Select region
#region= 'Israel_nan'
#region='Brazil_nan'
#region='Unites States_nan'
region='India_nan'
#region='China_Hubei'

evaluation=pd.DataFrame(columns=['region','mse','rmse','mae'])
place=0
for i in range(1,len(t)):
  if(t.iloc[i,1] is not t.iloc[i-1,1]):
    ex=np.array(t.iloc[i-len(ts):i,10])
    pred=np.array(t.iloc[i-len(ts):i,2])
    evaluation=evaluation.append({'region': t.iloc[i-1,1], 'mse': np.power((ex - pred),2).mean(),'rmse':sqrt(mean_squared_error(ex,pred)),'mae': (abs(ex - pred)).mean()}, ignore_index=True)
p=t[t['region']==region][['date','region','confirmed','kalman_prediction']]
p=p.rename(columns = {'confirmed':'confirmed'})
p.iloc[len(p)-1,2]=None
p=p.set_index(['date'])
p.iloc[30:,1:].plot(marker='o',figsize=(16,8)).set_title('Kalman Prediction - {}'.format(p.iloc[0,0]))
print(evaluation[evaluation['region']==p.iloc[0,0]])


* Displaying the output of the code in the above cell.

* Choosing the region as India and displaying the results.

In [ ]:
from ipywidgets import Image
f = open("../input/pictures/g2.png", "rb")
image = f.read()
Image(value=image)

* By doing the below mentioned steps, we are trying to find out the importance of each variables, used for the the prediction, using random forest classifier provied by h20 package.

* You can find further information regarding h20 here: http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/intro.html

* Find more information on linear regression here: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

# Importing the necessary packages for finding out the variable importance

In [ ]:
!pip install h2o
import h2o
from h2o.estimators import H2ORandomForestEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.grid.grid_search import H2OGridSearch
h2o.init(min_mem_size='8G')
import numpy as np
from sklearn.linear_model import LinearRegression

# Setting the dates and giving more weight to the recent dates.

In [0]:
train=train.fillna(0) 
train_df=train[train['date']<'2020-02-17']
boots=train_df[train_df['date']>='2020-02-14'] #to give more weight for recent days
train_df=train_df.append([boots[boots['date']>='2020-02-14']]*1000,ignore_index=True)
train_df_hubei=train_df[train_df['region']=='China_Hubei']
test=train[train['date']>='2020-02-17']
test=test[test['date']<'2020-02-19']

* Taking into account, all the variables regarding the variable importance check.

In [0]:
x_col=[#'region',
            '1_day_change', '3_day_change','7_day_change',
             '1_day_change_rate', 
            '3_day_change_rate',
             '7_day_change_rate', 
            'last_day', 'kalman_prediction','infected_rate', 'min', 'max'
          ]

* Here we are using the kalmin filtered inputs for our linear regression model, which are much accurate due to the usage of the immediate previous time step.

In [0]:
x=train_df[x_col]
y=train_df['confirmed']
reg = LinearRegression().fit(x,y)

pred2=reg.predict(test[x_col]); pred2=pd.DataFrame(pred2); pred2=round(pred2)
pred2['confirmed']=test['confirmed'].values; pred2['date']=test['date'].values; pred2['region']=test['region'].values
#pred2.iloc[:55]

In [0]:
train_h20 = h2o.H2OFrame(train_df)
train_h20_hubei = h2o.H2OFrame(train_df_hubei) # different model for Hubei
training_columns = ['region','1_day_change', '3_day_change', '7_day_change', '1_day_change_rate', '3_day_change_rate',
                    '7_day_change_rate', 'last_day', 'kalman_prediction','infected_rate', 'min', 'max'
                   ]                 
# Output parameter train against input parameters
response_column = 'confirmed'

# model = H2ORandomForestEstimator(ntrees=300, max_depth=12)
# model.train(x=training_columns, y=response_column, training_frame=train_h20)
model_hubei = H2ORandomForestEstimator(ntrees=300, max_depth=12)
model_hubei.train(x=training_columns, y=response_column, training_frame=train_h20_hubei)

test_h20 = h2o.H2OFrame(test)
#test_h20_hubei = h2o.H2OFrame(test_hubei)

Feature importance for the Hubei model using RF

In [0]:
model_hubei.varimp(True).iloc[:,:] # Feature importance for Hubei Model RF

* The correlation matrix, for showing the variable importance.

In [0]:
from string import ascii_letters
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white")
# Compute the correlation matrix
corr = train.iloc[:,2:].corr()
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=np.bool))
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))
# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.9, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
print ('Correlation Matrix')

* Displaying the output of the code mentioned in the above cell

In [ ]:
from ipywidgets import Image
f = open("../input/pictures/rf.png", "rb")
image = f.read()
Image(value=image)

In [0]:

print('Correlation To Confirmed') 
print (corr.confirmed)

* A graph showing the variation of minimum and maximum temperatures in the india Region for example purpose.

In [0]:
import matplotlib.pyplot as plt
p=train[['date','region','min','max']].set_index('date')
p=p[p['region']=='India_nan']
p.iloc[:37,:].plot(marker='*',figsize=(12,4),color=['#19303f','#cccc00']).set_title('Daily Min/Max Temperature - India',fontdict={'fontsize': 20})


* Displaying the output of the code mentioned in the above cell.

In [ ]:
from ipywidgets import Image
f = open("../input/pictures/mm.png", "rb")
image = f.read()
Image(value=image)

* A table showing more infected areas vs temperature.

In [0]:
avg_temp=train[['region','confirmed','min','max']]  # from 17-02-20
avg_temp=avg_temp.groupby(by='region').mean()
avg_temp=avg_temp.sort_values('confirmed',ascending=False)
print( 'Most infected Areas Avg Temperature')

## **X Day ahead prediction using kalman filter**

* Using the R interface again.


In [0]:

%%R
install.packages('reshape')

* Find more information regarding readr here: https://cran.r-project.org/package=readr

* Find more information regarding metrics here: https://cran.r-project.org/web/packages/Metrics/Metrics.pdf

In [0]:
%%R
require(pracma)
require(Metrics)
require(readr)
library(reshape)
all<- read_csv("..input/populationv/ts_r2.csv")
all$X1<-NULL
for (i in 1:30) { # Set i days prediction
  if( i>1) {all<-all_new}
  date<-all[,1]
  date[nrow(date) + 1,1] <-all[nrow(all),1]+1
  pred_all<-NULL
  for (n in 2:ncol(all)-1) {
    Y<-ts(data = all[n+1], start = 1, end =nrow(all)+1)  
    sig_w<-0.01
    w<-sig_w*randn(1,100) # acceleration which denotes the fluctuation (Q/R) rnorm(100, mean = 0, sd = 1)
    sig_v<-0.01
    v<-sig_v*randn(1,100)  
    t<-0.45
    phi<-matrix(c(1,0,t,1),2,2)
    gama<-matrix(c(0.5*t^2,t),2,1)
    H<-matrix(c(1,0),1,2)
    #Kalman
    x0_0<-p0_0<-matrix(c(0,0),2,1)
    p0_0<-matrix(c(1,0,0,1),2,2)
    Q<-0.01
    R<-0.01
    X<-NULL
    X2<-NULL
    pred<-NULL
    for (i in 0:nrow(all)) {
      namp <-paste("p", i+1,"_",i, sep = "")
      assign(namp, phi%*%(get(paste("p", i,"_",i, sep = "")))%*%t(phi)+gama%*%Q%*%t(gama))
      namk <- paste("k", i+1, sep = "")
      assign(namk,get(paste("p", i+1,"_",i, sep = ""))%*%t(H)%*%(1/(H%*%get(paste("p", i+1,"_",i, sep = ""))%*%t(H)+R)))
      namx <- paste("x", i+1,"_",i, sep = "")
      assign(namx,phi%*%get(paste("x", i,"_",i, sep = "")))
      namE <- paste("E", i+1, sep = "")
      assign(namE,Y[i+1]-H%*%get(paste("x", i+1,"_",i, sep = "")))
      namx2 <- paste("x", i+1,"_",i+1, sep = "")
      assign(namx2,get(paste("x", i+1,"_",i, sep = ""))+get(paste("k", i+1, sep = ""))%*%get(paste("E", i+1, sep = "")))
      namp2 <- paste("p", i+1,"_",i+1, sep = "")
      assign(namp2,(p0_0-get(paste("k", i+1, sep = ""))%*%H)%*%get(paste("p", i+1,"_",i, sep = "")))
      X<-rbind(X,get(paste("x", i+1,"_",i,sep = ""))[1])
      X2<-rbind(X2,get(paste("x", i+1,"_",i,sep = ""))[2])
      if(i>2){
        remove(list=(paste("p", i-1,"_",i-2, sep = "")))
        remove(list=(paste("k", i-1, sep = "")))
        remove(list=(paste("E", i-1, sep = "")))
        remove(list=(paste("p", i-2,"_",i-2, sep = "")))
        remove(list=(paste("x", i-1,"_",i-2, sep = "")))
        remove(list=(paste("x", i-2,"_",i-2, sep = "")))}
    } 
    pred<-NULL
    pred<-cbind(Y,X,round(X2,4))
    pred<-as.data.frame(pred)
    pred$region<-colnames(all[,n+1])
    pred$date<-date$date
    pred$actual<-rbind(0,(cbind(pred[2:nrow(pred),1])/pred[1:nrow(pred)-1,1]-1)*100)
    pred$predict<-rbind(0,(cbind(pred[2:nrow(pred),2])/pred[1:nrow(pred)-1,2]-1)*100)
    pred$pred_rate<-(pred$X/pred$Y-1)*100
    pred$X2_change<-rbind(0,(cbind(pred[2:nrow(pred),3]-pred[1:nrow(pred)-1,3])))
    pred_all<-rbind(pred_all,pred)
  }
  pred_all<-cbind(pred_all[,4:5],pred_all[,1:3])
  names(pred_all)[5]<-"X2"
  pred_all<-pred_all[,1:5]
       
pred_all_today=pred_all[with( pred_all, order(region, date)), ]
all_new=all
#all_new[nrow(all_new),1]<-all_new[nrow(all),1]+1
temp<-with(pred_all_today, pred_all_today[date == all[nrow(all),1]+1, ])
temp<-cbind(temp[,1:2],temp[,4])
temp2<-reshape(temp, direction = "wide", idvar = "date", timevar = "region")
rand_num<-runif(ncol(temp2)-1, 0.9, 1.05)
temp2[,2:ncol(temp2)]<-temp2[,2:ncol(temp2)]*rand_num
colnames(temp2)=colnames(all_new)
all_new<-rbind(all_new,temp2)
all_new[,2:ncol(all_new)]<-round(all_new[,2:ncol(all_new)])
for (i in 2:ncol(all_new)) {
  all_new[nrow(all_new),i]=max(all_new[nrow(all_new)-1,i],all_new[nrow(all_new),i])}
}

In [0]:
all_new=%R all_new

* Converting the date column values dtype to datetime format.

* Find out more information here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html

In [0]:
all_new['date']=pd.to_datetime(all_new['date'],unit='d')

In [0]:
# Select region
region=['date','China_Anhui', 'China_Beijing',
       'China_Chongqing', 'China_Fujian', 'China_Gansu',
       'China_Guangdong', 'China_Guangxi', 'China_Guizhou',
       'China_Hainan', 'China_Hebei', 'China_Heilongjiang', 'China_Henan','China_Hunan', 
       'China_Jiangsu', 'China_Jiangxi', 'China_Jilin', 'China_Liaoning',
       'China_Ningxia', 'China_Qinghai', 'China_Shaanxi',
       'China_Shandong', 'China_Shanghai', 'China_Shanxi',
       'China_Sichuan',  'China_Xinjiang',
       'China_Yunnan', 'China_Zhejiang', 
        'Hong Kong_Hong Kong','Japan_nan','Others_Diamond Princess cruise ship']
#p_kalman=all_new[region]
#p=all_new
#p.iloc[len(p)-1,2]=None
#p_kalman=p_kalman.set_index(['date'])
#p_kalman.iloc[:,:].plot(marker='o',figsize=(24,14)).set_title('Kalman Prediction')

p_kalman2=all_new[['date','India_nan']]
p_kalman2=p_kalman2.set_index(['date'])
p_kalman2.iloc[:,:].plot(marker='o',figsize=(16,8)).set_title('Kalman Prediction - Select Region to Change - {}'.format(p_kalman2.columns[0]))


* Displaying the output of the above mentioned code.

In [ ]:
from ipywidgets import Image
f = open("../input/pictures/30day.png", "rb")
image = f.read()
Image(value=image)

In [0]:
all_new.info()

All the countries having the predictions

# **Iterative Regression**

* Fitting the regression model to our data.

In [0]:
t_iter=all_new.set_index(['date'])
t_iter=t_iter.stack().reset_index(name='confirmed')
t_iter.columns=['date', 'region','confirmed']
t_iter['date']=pd.to_datetime(t_iter['date'] ,errors ='coerce')
t_iter=t_iter.sort_values(['region', 'date'])

t_iter=t_iter.reset_index(drop=True)
for i in range(1,len(t_iter)+1):
  if(t_iter.iloc[i,1] is not t_iter.iloc[i-1,1]):
    t_iter.loc[len(t_iter)+1] = [t_iter.iloc[i-1,0]+ pd.DateOffset(1),t_iter.iloc[i-1,1], 0] 
t_iter=t_iter.sort_values(['region', 'date'])
t_iter=t_iter.reset_index(drop=True)

t_iter['1_day_change']=t_iter['3_day_change']=t_iter['7_day_change']=t_iter['1_day_change_rate']=t_iter['3_day_change_rate']=t_iter['7_day_change_rate']=t_iter['last_day']=0
for i in range(1,len(t_iter)):
  if(t_iter.iloc[i,1] is t_iter.iloc[i-2,1]):
    t_iter.iloc[i,3]=t_iter.iloc[i-1,2]-t_iter.iloc[i-2,2]
    t_iter.iloc[i,6]=(t_iter.iloc[i-1,2]/t_iter.iloc[i-2,2]-1)*100
    t_iter.iloc[i,9]=t_iter.iloc[i-1,2]
  if(t_iter.iloc[i,1] is t_iter.iloc[i-4,1]):
    t_iter.iloc[i,4]=t_iter.iloc[i-1,2]-t_iter.iloc[i-4,2]
    t_iter.iloc[i,7]=(t_iter.iloc[i-1,2]/t_iter.iloc[i-4,2]-1)*100
  if(t_iter.iloc[i,1] is t_iter.iloc[i-8,1]):
    t_iter.iloc[i,5]=t_iter.iloc[i-1,2]-t_iter.iloc[i-8,2]
    t_iter.iloc[i,8]=(t_iter.iloc[i-1,2]/t_iter.iloc[i-8,2]-1)*100
t_iter=t_iter.fillna(0)  

# t_iter=t_iter.merge(temp[['date','region', 'X']],how='left',on=['date','region'])
# t_iter=t_iter.rename(columns = {'X':'kalman_prediction'}) 
t_iter=t_iter.replace([np.inf, -np.inf], 0)
t_iter['kalman_prediction']=round(t_iter['confirmed'])
test_iter=t_iter.merge(confirmed[['region',' Population ']],how='left',on='region')
test_iter=test_iter.rename(columns = {' Population ':'population'})
test_iter['population']=test_iter['population'].str.replace(r" ", '')
test_iter['population']=test_iter['population'].str.replace(r",", '')
test_iter['population']=test_iter['population'].fillna(1)
test_iter['population']=test_iter['population'].astype('int32')
test_iter['infected_rate'] =test_iter['last_day']/test_iter['population']*10000
test_iter=test_iter.merge(w,how='left',on=['date','region'])
#test_iter=test_iter.sort_values(['region', 'date'])
test_iter_temp=test_iter[np.isnan(test_iter['min'])]
test_iter_temp=test_iter_temp.drop(columns=['min', 'max'])
test_iter_temp=test_iter_temp.merge(w_forecast,how='left',on=['date','region'])
test_iter=test_iter.dropna()
test_iter=test_iter.append(test_iter_temp)
test_iter=test_iter.sort_values(['region', 'date'])
### fill missing weather 
for i in range(0,len(test_iter)):
  if(np.isnan(test_iter.iloc[i,13])):
    if(test_iter.iloc[i,1] is test_iter.iloc[i-1,1]):
      test_iter.iloc[i,13]=test_iter.iloc[i-1,13]+abs(test_iter.iloc[i-1,13]*.01)
      test_iter.iloc[i,14]=test_iter.iloc[i-1,14]+abs(test_iter.iloc[i-1,14]*.01)

* Filling the missing values with 0 values.

In [0]:
test_iter=test_iter.fillna(0) 

In [0]:
pred=reg.predict(test_iter[x_col]); pred=pd.DataFrame(pred); pred.columns = ['prediction'];pred=round(pred)
pred['confirmed']=test_iter['confirmed'].values; pred['date']=test_iter['date'].values; pred['region']=test_iter['region'].values
for i in range(1,len(pred)):
    if(pred.iloc[i,3] is pred.iloc[i-1,3]):
      if(pred.iloc[i,0]<pred.iloc[i-1,1]):
        pred.iloc[i,0]=pred.iloc[i-1,1]
pred = pred.pivot_table(index='date',columns='region',values='prediction') # pivot pred df

* Displaying the information about the datatypes present in different columns of the pred-DataFrame.

In [0]:
pred.info()

* Now saving the above obtained predictions for using them in the npi prediction step.

In [0]:
d = pred
d = d.T
d.to_csv(r'../input/tsv/pred.csv')


# Now, we go into step 2 wherin we are going to try and predict the best applicable npis for a particular situation.

* Again as a prelimnary step, we will be importing all the necessary packages, libraries and algorithms.

* For more details regarding the algorithms and their usage on your own dataset, please find it here: https://scikit-learn.org/

In [ ]:
# Importing the libraries
import numpy as np
import pandas as pd
import datetime 
import calendar 
# data visualization
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import style
from sklearn.preprocessing import StandardScaler

# Algorithms
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.tree import ExtraTreeClassifier

# Importing our cleaned and modified dataset.

* Sources:

* https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset 

* https://www.kaggle.com/davidoj/covid19-national-responses-dataset

* https://www.kaggle.com/fernandol/countries-of-the-world

Also the prediction-values which I have saved in the last phase of step 1.

Some details regarding the columns of mastersheet prediction are:

* Each row is an entry/instance of a particular Npi getting implemented.

* Country: This column represents the country to which the entry belongs to.

* Net migration: The net migration rate is the difference between the number of immigrants (people coming into an area) and the number of emigrants (people leaving an area) throughout the year.

* Population density: Population density is the number of individuals per unit geographic area, for example, number per square meter, per hectare, or per square kilometer.

* Sex Ratio: The sex ratio is the ratio of males to females in a population.

* Population age-distribution: Age distribution, also called Age Composition, in population studies, the proportionate numbers of persons in successive age categories in a given population. (0-14yrs/60+yrs %)

* Health physicians per 1000 population: Number of medical doctors (physicians), including generalist and specialist medical practitioners, per 1 000 population.

* Mobile cellular subscription per 100 inhabitants: Mobile cellular telephone subscriptions are subscriptions to a public mobile telephone service that provide access to the PSTN using cellular technology.

* Active on day: The number of active cases of covid19 infections in that particular country on the day it was implemented.

* Sevenday, twelveday and thirty day predictions are for active cases from the date it was implemented.

* And the date-implemented is converted to whether it was a week-day or a week-end to make it usable for training.

* The last column represents the category to which the npi that implemented belonged to.

Category 1: Public -health measures and social-distancing.

Category 2: Social-economic measures and movement-restrictions.

Category 3: Partial/complete lockdown.


* To categorise the npis we followed a 5 step analysis:

* Step 1: We choosed 6 different countries that have implemented atleast one of the above-mentioned npis.

* Step 2: We had chosen a particular date wherein one of the npi was implemented.

* Step 3: From that date (chosen) we had calculated 5day, 8day, 12day growth rate in the number of confirmed cases in that country.

* Step 4: According to: 1) https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4327893/
    2) https://www.worldometers.info/coronavirus/coronavirus-incubation-period/
    we took a reference that, over 50% of the people who are affected on day1 show symptoms by day5, over 30% of the people affected on           day1 show symptoms by day8 and the last 20% start showing symptoms by day12. Assuming that, they get a checkup as soon as they are             showing symptoms, we had calculated a cumilative growth rate.
* Step 5: This cumilative growth rate was not very accurate due to the population densities of the countries being different. So, we had normalised the obtained scores from step4 by the population densities. That gave us the following results.

* More information can be found here: https://archive.is/JNx03 and https://archive.is/UA3g14

* [ (896.4961042933885, 'CHINA', 'SOCIAL DISTANCING'),
*  (720.7571447424511, 'FRANCE', 'PUBLIC HEALTH MEASURES'),
*  (578.0345389562175, 'SPAIN', 'SOCIAL AND ECONOMIC MEASURES'),
*  (527.7087251438776, 'IRAN', 'MOV RESTRICTION'),
*  (484.1021819976962, 'ITALY', 'PARTIAL LOCKDOWN'),
*  (207.67676767676767, 'INDIA', 'COMPLETE LOCKDOWN')]

Ex: (Cumilative growthrate(normalised), Country Name, Measure-taken)

* So the above analysis shows the decreasing order of growth rates, however this is not very accurate due to various other reasons, but this gives a rough estimate of the effectiveness/strength of the npis.

* With this categorisation in mind, let's go the next step.

So, the dataset that we used is: https://www.kaggle.com/davidoj/covid19-national-responses-dataset

In [ ]:
dataset = pd.read_csv("../input/mastersheetpred/mastersheetprediction.csv")

* Noting down the information of the dataset for a quick look,

In [ ]:
dataset.info()

# There are 54 countries in the dataset, which have more than 1000cases at the time of creating this notebook.

In [ ]:
dataset.Country.unique()

# Now extracting the input-features and the target by df.iloc function.
* For more information regarding the df.iloc, please use this link: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html

In [ ]:
X = dataset.iloc[:, 0:13].values# input feature
y = dataset.iloc[:, 13].values# label/target

* Checking the shape by using the numpy.ndarray.shape function.

* Find more information here: https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.shape.html

In [ ]:
X.shape, y.shape

* The next step is encoding the categorical data, this was done using labelencoder and one-hot encoder.

* You can find more information regarding the sklearn-preprocessing library here: http://scikit-learn.org/stable/modules/preprocessing.html

* Instead of one-hot encoder, we used using ColumnTransformer here: http://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
#encoding the X
ct = ColumnTransformer([("Country", OneHotEncoder(), [0])], remainder = 'passthrough')
X = ct.fit_transform(X)
X = X[:, 0:]

# Now we split the dataset into train and test set.

* We will used train_test_split for this part, find more information here: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 0)

# We would be feature scaling the inputs.

* We used standard StandardScaler function from sklearn's preprocessing library.

* Find more information here: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train.toarray())
X_test = sc.transform(X_test.toarray())

# Checking the frequencies of our categories in the target array-(y)

* We use the np.unique() function, find more information here: https://docs.scipy.org/doc/numpy/reference/generated/numpy.unique.html

In [ ]:
np.unique(y, return_counts=True)

# Class-imbalance problem.

* We notice that there is a class-imbalance problem. The caegory 3 is a minority class which is only 2% of the dataset.

* To tackle this problem we have used smote technique(over-sampling technique as we have less number of examples), find more information here: https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.SMOTE.html


In [ ]:
!pip install smote-variants

In [ ]:
import smote_variants as sv

oversampler= sv.MulticlassOversampling(sv.distance_SMOTE())
# X_samp and y_samp contain the oversampled train-dataset
X_samp, y_samp= oversampler.sample(X_train, y_train)
#oversampling the test dataset
X_samptest, y_samptest= oversampler.sample(X_test, y_test.ravel())

In [ ]:
y_samp.shape, y_samptest.shape

# The problem of multilabel classification.

* Multi-label classification is a generalization of multiclass classification, which is the single-label problem of categorizing instances into precisely one of more than two classes.

* Our dataset suffers from multilabel classification problem, this is due to that many countries have implemented npis that belong to more than one category in a single day, this makes an entry which have all the input variables same but the target-category is different. 

* This makes it a difficult problem with the existing features.

* Drawback: We did not have any strong differentiating feature in our dataset.

# To tackle this problem of multilabel classfication problem to an extent we used various multilearn models from sklearn.

# To use the multioutputclassifier, we encoded the category column.

In [ ]:
yovs = pd.read_csv("../input/oversample/oversampletargets.csv")
yovs = yovs.iloc[:,1:]
yovs

* We use the multi-ouput classifier using the randomforest as the base-algorithm.

* Find more information here: http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html

In [ ]:
#multi-output classifier and the random-forest classifier.
from sklearn.multioutput import MultiOutputClassifier
random_forest = RandomForestClassifier(n_estimators=1000, criterion = 'entropy')#with single column targets
random_forest.fit(X_samp, y_samp)

Y_pred = random_forest.predict(X_samp)

multi_target_forest = MultiOutputClassifier(random_forest, n_jobs=-1)
Ypred = multi_target_forest.fit(X_samp, yovs).predict(X_samp)
multi_target_forest.score(X_samp, yovs)
acc_multi = round(multi_target_forest.score(X_samp, yovs) * 100, 2)
print(acc_multi,"trainset-multi-output")

random_forest.score(X_samp, y_samp)
acc_random_forest = round(random_forest.score(X_samp, y_samp) * 100, 2)
print(acc_random_forest,"trainset-Randomforest")

* Hamming loss for the predictions on train set by randomforest classifier.

In [ ]:
from sklearn.metrics import hamming_loss
print("Hamming loss = ",hamming_loss(y_samp,Y_pred))

# We used the pipeline technique from sklearn.

* Find more information here: http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

* We used the NeighborhoodComponentsAnalysis and  KNeighborsClassifier with pipeline.

In [ ]:
from sklearn.neighbors import (NeighborhoodComponentsAnalysis, KNeighborsClassifier)
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

nca = NeighborhoodComponentsAnalysis(random_state=10)
knn = KNeighborsClassifier(n_neighbors=1)
nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
nca_pipe.fit(X_samp, y_samp)

print(nca_pipe.score(X_samp, y_samp)*100, "%")

# Finding the oob score using RF classifier.

* Find more information here: https://en.wikipedia.org/wiki/Out-of-bag_error

* We are finding the oob-score for our train set.

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100, criterion = 'entropy', oob_score = True)
random_forest.fit(X_samp, y_samp)
Y_prediction = random_forest.predict(X_samp)

random_forest.score(X_samp, y_samp)

acc_random_forest2 = round(random_forest.score(X_samp, y_samp) * 100, 2)
print(round(acc_random_forest2,2,), "%")

print("oob score:", round(random_forest.oob_score_, 4)*100, "%")

# We used Cross validation also and calculated mean accuracies.

* For more information, refer this: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

In [ ]:
from sklearn.model_selection import cross_val_score
rf = RandomForestClassifier(n_estimators=100, criterion = 'entropy')
scores = cross_val_score(rf, X_samp, y_samp, cv=10, scoring = "accuracy")
print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())

# KNeighborsClassifier

* Find more information here: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 1) 
knn.fit(X_samp, y_samp)  
Y_pred = knn.predict(X_samp)  
acc_knn = round(knn.score(X_samp, y_samp) * 100, 2)
print(acc_knn,"trainset")

In [ ]:
from sklearn.metrics import hamming_loss
print("Hamming loss-trainset = ",hamming_loss(y_samp,Y_pred))

# Decision tree-classifier
* For more information: http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [ ]:
decision_tree = DecisionTreeClassifier() 
decision_tree.fit(X_samp, y_samp)  
Y_pred = decision_tree.predict(X_samp)  
acc_decision_tree = round(decision_tree.score(X_samp, y_samp) * 100, 2)
print(acc_decision_tree,"trainset")

The hamming loss has gone down.

In [ ]:
from sklearn.metrics import hamming_loss
print("Hamming loss-trainset = ",hamming_loss(y_samp,Y_pred))

# Hyperparameter tuning using GridsearchCV

* We tuned the hyperparameters of the random-forest classifier using gridsearchcv.

* Find more information here: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [ ]:
param_grid = { "criterion" : ["gini", "entropy"], "min_samples_leaf" : [1, 5, 10, 25, 50, 70], "min_samples_split" : [2, 4, 10, 12, 16, 18, 25, 35], "n_estimators": [100, 400, 700, 1000, 1500]}
from sklearn.model_selection import GridSearchCV, cross_val_score
rf = RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=1, n_jobs=-1)
clf = GridSearchCV(estimator=rf, param_grid=param_grid, n_jobs=-1)
clf.fit(X_samp, y_samp)
clf.best_params_

* I'm not executing the above cell, because it will take a huge time to get completed, rather the output is displayed below.

{'criterion': 'gini',
 'min_samples_leaf': 10,
 'min_samples_split': 2,
 'n_estimators': 100}

* Retraining the Rf with the above mentioned hyperparameters.

In [ ]:
random_forest = RandomForestClassifier(criterion = "gini",    #the best parameters for the rf classifier
                                       min_samples_leaf = 10, 
                                       min_samples_split = 2,   
                                       n_estimators=100, 
                                       max_features='auto', 
                                       oob_score=True, 
                                       random_state=1, 
                                       n_jobs=-1)

random_forest.fit(X_samp, y_samp)
Y_prediction = random_forest.predict(X_samptest)

random_forest.score(X_samp, y_samp)

print("oob score:", round(random_forest.oob_score_, 4)*100, "%")

# Implementing classifier chain technique as it tackles multilabel classification problems.

* For more information: http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.ClassifierChain.html

In [ ]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.ensemble import RandomForestClassifier

# initialize LabelPowerset multi-label classifier with a RandomForest
classifier = ClassifierChain(
    classifier = RandomForestClassifier(n_estimators=200),
    require_dense = [False, True]
)

# train
classifier.fit(X_samp, y_samp)

# predict
predictions = classifier.predict(X_samp)
pred2 = classifier.predict(X_samptest)

In [ ]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy-train:",metrics.accuracy_score(y_samp.reshape(-1,1), predictions.todense())*100, "%")
print("Accuracy-test:",metrics.accuracy_score(y_samptest.reshape(-1,1), pred2.todense())*100, "%")

# Using the onevsrest classifier.

* For more details: http://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html

In [ ]:
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
pred = OneVsRestClassifier(RandomForestClassifier(n_estimators=200, class_weight='balanced')).fit(X_samp, y_samp).predict(X_samp)
pred2 = OneVsRestClassifier(RandomForestClassifier(n_estimators=200, class_weight='balanced')).fit(X_samp, y_samp).predict(X_samptest)

In [ ]:
print("Accuracy-train:",metrics.accuracy_score(y_samp, pred)*100, "%")
print("Accuracy-test:",metrics.accuracy_score(y_samptest, pred2)*100, "%")

* This is by far the best accuracy we have got on the train and test set.

# Next, RadiusNeighborsClassifier with radius =1.

* It is considered to be a lazy algorithm.

* This gives the best accuracy as this fits the best to our dataset. This is because the training-examples we used for classification are very closed to each other in the feature space.

* For more information: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.RadiusNeighborsClassifier.html

In [ ]:
from sklearn.neighbors import RadiusNeighborsClassifier
neigh = RadiusNeighborsClassifier(radius=1, weights='distance',algorithm='brute')
neigh.fit(X_samp, y_samp)
ypredu = neigh.predict(X_samp)

In [ ]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy-train:",metrics.accuracy_score(y_samp, ypredu)*100, "%")

In [ ]:
from sklearn.metrics import hamming_loss
print("Hamming loss = ",hamming_loss(y_samp,ypredu))

Find more about the jaccard_score: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.jaccard_score.html

In [ ]:
from sklearn.metrics import jaccard_score
js= jaccard_score(y_samp, ypredu, average=None)
js

Find more about the classification report: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

In [ ]:
from sklearn.metrics import classification_report
target_names = ['class 1', 'class 2', 'class 3']
print(classification_report(y_samp, ypredu, target_names=target_names))

* We plot the confusion matrix to see what's actually happening.

* Find more information here: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

In [ ]:

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
def plot_confusion_matrix(cm, classes, normalized=True, cmap='bone'):
    plt.figure(figsize=[7, 6])
    norm_cm = cm
    if normalized:
        norm_cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        sns.heatmap(norm_cm, annot=cm, fmt='g', xticklabels=classes, yticklabels=classes, cmap=cmap)
        plt.savefig('confusion-matrix.png')

In [ ]:
from sklearn.metrics import confusion_matrix
target_names = ['class 1', 'class 2', 'class 3']
cm = confusion_matrix(y_samp, ypredu)

In [ ]:
plot_confusion_matrix(cm, classes=target_names)

# The drawbacks of our approach.

* This above-mentioned approach has many drawbacks, one of them is an incomplete dataset.

* There are no good-differentiating features in the dataset.

* In our approach we are not able to decide the effectiveness and a go-to plan of action for deploying npis.

* All the data-points are very-similar to one-another, hence it is being difficult for the algorithm to learn.

# Further work:

* There could be a set of strong differentiating features in the dataset, which will make the generalization easy.

* There can be further categorisation of npis for better implementation of them.

* The dataset can also be combined with economic parameters further, to understand the economic feasibilty of the npi-implementation.

* It can further be used to predict the decrease in growth rates, once an npi is implemented to further note the real-time effectiveness of the npis in a particular demographic

Note: This work is inspired from many data science resources. Any traces of replications, which may appear, is purely co-incidental.